In [1]:
# https://www.tensorflow.org/tutorials/text/text_generation?hl=ko
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('텍스트의 길이: {}자'.format(len(text)))

텍스트의 길이: 1115394자


In [5]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
vocab = sorted(set(text))
print ('고유 문자수 {}개'.format(len(vocab)))

고유 문자수 65개


In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [9]:
print ('{} ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- 문자들이 다음의 정수로 매핑되었습니다 ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [10]:
print(text_as_int)

[18 47 56 ... 45  8  0]


In [11]:
seq_length = 100
examples_per_epoch = len(text)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [14]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>


In [15]:
for input_example, target_example in  dataset.take(1):
    print ('입력 데이터: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('타깃 데이터: ', repr(''.join(idx2char[target_example.numpy()])))

입력 데이터:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
타깃 데이터:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [16]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("{:4d}단계".format(i))
    print("  입력: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  예상 출력: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

   0단계
  입력: 18 ('F')
  예상 출력: 47 ('i')
   1단계
  입력: 47 ('i')
  예상 출력: 56 ('r')
   2단계
  입력: 56 ('r')
  예상 출력: 57 ('s')
   3단계
  입력: 57 ('s')
  예상 출력: 58 ('t')
   4단계
  입력: 58 ('t')
  예상 출력: 1 (' ')


In [17]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [18]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [19]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [20]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [22]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (배치 크기, 시퀀스 길이, 어휘 사전 크기)")

(64, 100, 65) # (배치 크기, 시퀀스 길이, 어휘 사전 크기)


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [24]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [25]:
sampled_indices

array([11, 23, 62, 60, 26, 62, 38,  0, 27, 28, 23, 49, 36, 31, 12,  0, 33,
       43, 60, 19, 64, 15,  3,  9, 33, 46,  8, 63, 64, 46, 16, 41, 18, 41,
       14,  6, 41, 48, 25, 61, 23, 59, 28, 24, 16, 58, 18, 30, 40, 34, 22,
       64, 31, 33, 54, 46, 42,  5, 43,  9, 47,  3, 17, 24, 32, 44, 23,  2,
       45,  3,  5, 14, 55, 27, 10, 16,  8,  6, 60,  0, 26, 53,  8, 51, 18,
       44, 15,  7, 31, 54,  1, 58, 57, 60, 25, 49, 47, 14, 52, 32])

In [26]:
print("입력: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("예측된 다음 문자: \n", repr("".join(idx2char[sampled_indices ])))

입력: 
 'in books:\nIf you accept them, then their worth is great.\n\nBAPTISTA:\nLucentio is your name; of whence'

예측된 다음 문자: 
 ";KxvNxZ\nOPKkXS?\nUevGzC$3Uh.yzhDcFcB,cjMwKuPLDtFRbVJzSUphd'e3i$ELTfK!g$'BqO:D.,v\nNo.mFfC-Sp tsvMkiBnT"


In [29]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("예측 배열 크기(shape): ", example_batch_predictions.shape, " # (배치 크기, 시퀀스 길이, 어휘 사전 크기")
print("스칼라 손실:          ", example_batch_loss.numpy().mean())

예측 배열 크기(shape):  (64, 100, 65)  # (배치 크기, 시퀀스 길이, 어휘 사전 크기
스칼라 손실:           4.175736


In [30]:
model.compile(optimizer='adam', loss=loss)

In [31]:
# 체크포인트가 저장될 디렉토리
checkpoint_dir = './training_checkpoints'
# 체크포인트 파일 이름
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [32]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
  3/172 [..............................] - ETA: 4:11 - loss: 4.0200

KeyboardInterrupt: 

In [33]:
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

AttributeError: 'NoneType' object has no attribute 'endswith'

## Prediction Loop

In [38]:
def generate_text(model, start_string):
    num_generate = 1000
    
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    
    temperature = 1.0
    
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return (start_string + ''.join(text_generated))

In [39]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: ?BB&ynD?I:?WJFkzZaw
Q$Mqi.v'bgrv?lrZrKPFQExE:VD;EDVJuQFYearwMH3GON:QCEzVZvlL:uEXZPWKziabWDVy LWtqR?kfJKfx
vSmAeY'JFVGMAtNS;PN?dUqNEKBVykFxvqSiVm
Gc$H xX'xpWZYPcGv
VTirjNQxxMF!x!seHRwsVPlOLl.f?HMDW Eq;.$H!mnbTUuXONZf&vcUZWw.IuvJw&aluzDw,epZyDbZ;TiuPGs,Dsj!,XNTXag
:cRvnZAQD 3kkkCLq?VhbBlSEfqRcOXUwx-ENy?tDBfLp&YFt,N!YEQAN&itomvoQea?XWi:cszo'WIls; SPz;cRgAy:Di:'U,& u;&v-UqtwvfMP:YAOLlk HMMnDISgz$eoHoF
YHmd;O;!vAbQFKRdbwYwqd&L&$bz-,c:-YHAJdVW?, avEoVsPUj.ulS.t;lM$.DsCVtdlMKyaF-:?.tmmdrgWd'h$!3S'$YElaLYfbGd? C!FtUyrNIBtKlsAl?zAG
F3Exxt!MWHpRBmQwICDsBrH3dDPNhArorb;sin'lD?TBuuv jPjGl?RbhmyNmZf$XuwwDLyARdjW$oOV:LMRKpwluyAi xSjnvKeCybyr;q
gKUTmFaO:&lCQnJZ
pA,OBEFFVaF;mizuacBwn;PTlOoZhZezRconB:DFqnlBMcPisgHOLf;K,.IZS,U;KQDGYBZcmw$DgeN&!htPuKsbUzeGHURx' D.;.LFfEYD!izTHp Cz!hhub eg.uR-a!&QcCaATyBjhbYU&WecMQfhd?cASctMPJfF;EbZ;HH!g3-Lcj&bRCN joxM
$N$$So.PIqM$qIO3mmu!yyJoP;K3tM3CJKtTl'bAnZ3&r.AD.ytxPFFrnLETXq'UHkc!JvN3qoWQAJA
GjWNVqTar !A'JC q;?'uGyRh.uEb:'ml;iokkCbka:;FvGPd -NfvrBjpFcvogrw'BN,